# Data Distributions

## Setup

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

np.random.seed(707260)

# Point this to your directory
clinical_path = Path("../step2_prep_data/a_clinical_data/clinical_metrics/full_data.tsv")

In [ ]:
clinical_df = pd.read_csv(clinical_path, sep='\t').set_index('GRP')

In [ ]:
clinical_df

## Target Metrics

In [ ]:
# Generate a sub-dataframe to isolate the "labelling" metrics
mjoa_initial_key = 'mJOA initial'
mjoa_1year_key = 'mJOA 12 months'
hrr_key = 'HRR'
recovery_key = 'Recovery Class'
mjoa_df = clinical_df.loc[:, [mjoa_initial_key, mjoa_1year_key, hrr_key, recovery_key]]

# DCM Severity labelling (Initial)
mjoa_initial_class_key = "DCM Severity (initial)"
mjoa_df[mjoa_initial_class_key] = 'Severe'
mjoa_df.loc[mjoa_df[mjoa_initial_key] > 11, mjoa_initial_class_key] = 'Moderate'
mjoa_df.loc[mjoa_df[mjoa_initial_key] > 14, mjoa_initial_class_key] = 'Mild'
mjoa_df.loc[mjoa_df[mjoa_initial_key] > 17, mjoa_initial_class_key] = 'Healthy'

# DCM Severity labelling (1-Year)

mjoa_1year_class_key = "DCM Severity (1 year)"
mjoa_df[mjoa_1year_class_key] = 'Severe'
mjoa_df.loc[mjoa_df[mjoa_1year_key] > 11, mjoa_1year_class_key] = 'Moderate'
mjoa_df.loc[mjoa_df[mjoa_1year_key] > 14, mjoa_1year_class_key] = 'Mild'
mjoa_df.loc[mjoa_df[mjoa_1year_key] > 17, mjoa_1year_class_key] = 'Healthy'

# DCM Delta Labelling (Initial -> 1 Year)
mjoa_delta_key = 'mJOA Delta'
mjoa_df[mjoa_delta_key] = mjoa_df[mjoa_1year_key] - mjoa_df[mjoa_initial_key]

mjoa_delta_class_key = 'DCM Improvement (Initial -> 1 Year)'
mjoa_df[mjoa_delta_class_key] = 'Static'
mjoa_df.loc[mjoa_df[mjoa_delta_key] > 0, mjoa_delta_class_key] = 'Improved'
mjoa_df.loc[mjoa_df[mjoa_delta_key] < 0, mjoa_delta_class_key] = 'Declined'
mjoa_df

### mJOA

#### Prep 

In [ ]:
def plot_distributions(data, cmap, legend_elements, xlabel, title, mean_offset=0, flip_mean_rot=False):
    # Get the appropriate ranges for the data
    min_range = int(np.min(data))-1
    max_range = int(np.max(data))+1
    
    # Bin the data
    hist, bins = np.histogram(
        data, 
        np.array(range(min_range, max_range))+.1
    )
    
    # Generate the figure
    fig, ax = plt.subplots()
        
    # Iteratively color code the bars
    for t, c in cmap.items():
        mask = bins < t
        to_display = np.array(range(min_range, t))+0.5
        vals = hist[mask[:-1]]
        ax.bar(
            to_display, vals,
            width=1, color=c,
            align='edge',
            edgecolor='black'
        )
        
    # Add a mean line
    data_mean = np.mean(data)
    ax.axvline(data_mean, ls='--', c='black')
    if flip_mean_rot:
        ax.text(data_mean-0.5, ax.get_ylim()[1]-mean_offset, f"Mean ({data_mean:.4})", rotation=90)
    else:
        ax.text(data_mean+0.05, ax.get_ylim()[1]-mean_offset, f"Mean ({data_mean:.4})", rotation=-90)
        
    # Add in the legend
    ax.legend(handles=legend_elements)
    
    # Add in labels
    ax.set_xlabel(xlabel)
    ax.set_ylabel('Count')
    ax.set_title(title)
    
    # Return the figure and axis
    return fig, ax

In [ ]:
# Limits so that all plots have consistent range
xlim_min = int(np.min([*clinical_df['mJOA initial'], *clinical_df['mJOA 12 months']]))-1
xlim_max = int(np.max([*clinical_df['mJOA initial'], *clinical_df['mJOA 12 months']]))+1

ylim_min = 0
ylim_max = int(np.max([
    *np.histogram(clinical_df['mJOA initial'], np.array(range(xlim_min, xlim_max))+.1)[0],
    *np.histogram(clinical_df['mJOA 12 months'], np.array(range(xlim_min, xlim_max))+.1)[0]
]))+5

In [ ]:
# Generate the output path for plot files if it doesn't exist
mjoa_dist_out_path = Path('figures/mjoa_dist')
if not mjoa_dist_out_path.exists():
    mjoa_dist_out_path.mkdir(parents=True)

#### Initial

In [ ]:
# Color threshold map
severity_cmap = {
    18: 'blue',
    17: 'green',
    14: 'gold',
    11: 'red'
}

# Generate a custom legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='red', edgecolor='black', label='Severe'),
    Patch(facecolor='gold', edgecolor='black', label='Moderate'),
    Patch(facecolor='green', edgecolor='black', label='Mild'),
    Patch(facecolor='blue', edgecolor='black', label='Healthy'),
]

# Plot the data
fig, ax = plot_distributions(
    mjoa_df[mjoa_initial_key], severity_cmap, legend_elements,
    'mJOA', 'Pre-Surgical mJOA Scores', 20
)

# Plot the total number of each severity class as text
severity_counts = mjoa_df[mjoa_initial_class_key].value_counts()
ax.text(9, 15, f"({severity_counts['Severe']})", c='black', size=12, horizontalalignment='center')
ax.text(14, 44.5, f"({severity_counts['Moderate']})", c='black', size=12, horizontalalignment='center')
ax.text(16, 33, f"({severity_counts['Mild']})", c='black', size=12, horizontalalignment='center')
ax.text(18, 2.5, f"({severity_counts['Healthy']})", c='black', size=12, horizontalalignment='center')

# Save and show the result
fig.savefig(mjoa_dist_out_path / 'pre_treatment_mjoa.svg')
plt.show()

#### 1-Year

In [ ]:
# Color threshold map
severity_cmap = {
    18: 'blue',
    17: 'green',
    14: 'gold',
    11: 'red'
}

# Generate a custom legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='red', edgecolor='black', label='Severe'),
    Patch(facecolor='gold', edgecolor='black', label='Moderate'),
    Patch(facecolor='green', edgecolor='black', label='Mild'),
    Patch(facecolor='blue', edgecolor='black', label='Healthy'),
]

# Plot the data
fig, ax = plot_distributions(
    mjoa_df[mjoa_1year_key], severity_cmap, legend_elements,
    'mJOA', 'Pre-Surgical mJOA Scores', 20, flip_mean_rot=True
)

# Plot the total number of each severity class as text
severity_counts = mjoa_df[mjoa_1year_class_key].value_counts()
ax.text(9, 3, f"({severity_counts['Severe']})", c='black', size=12, horizontalalignment='center')
ax.text(13, 34, f"({severity_counts['Moderate']})", c='black', size=12, horizontalalignment='center')
ax.text(16, 60.5, f"({severity_counts['Mild']})", c='black', size=12, horizontalalignment='center')
ax.text(18.1, 37, f"({severity_counts['Healthy']})", c='black', size=12, horizontalalignment='center')

# Save and show the result
fig.savefig(mjoa_dist_out_path / 'post_treatment_mjoa.svg')
plt.show()

#### Change (Delta)

In [ ]:
# Define a new color scheme and legend for this new style of data
delta_cmap = {
    8: 'springgreen',
    0: 'white',
    -1: 'salmon'
}

delta_legend_elements = [
    Patch(facecolor='springgreen', edgecolor='black', label='Improved'),
    Patch(facecolor='white', edgecolor='black', label='No Change'),
    Patch(facecolor='salmon', edgecolor='black', label='Declined'),
]

xticks = (
    list(range(-8, 9, 2)),
    list(range(-8, 9, 2))
)

# Plot the data
fig, ax = plot_distributions(
    mjoa_df[mjoa_delta_key], delta_cmap, delta_legend_elements,
    "mJOA Change", 'Change in mJOA 1 Year Post-Surgery', 20, flip_mean_rot=True
)

# Plot the total number of each severity class as text
change_counts = pd.cut(
    mjoa_df[mjoa_delta_key], 
    [-20, -1, 0, 20], 
    labels=['Declined', 'No Change', 'Improved']
).value_counts()
ax.text(-4.5, 9, f"({change_counts['Declined']})", c='black', size=12, verticalalignment='center')
ax.text(-0.8, 39, f"({change_counts['No Change']})", c='black', size=12, verticalalignment='center')
ax.text(4, 32, f"({change_counts['Improved']})", c='black', size=12, verticalalignment='center')

# Save and show the result
fig.savefig(mjoa_dist_out_path / 'treatment_mjoa_delta.svg')
plt.show()

#### HRR

In [ ]:
from scipy.stats import gaussian_kde

# Plot the KDE distribution onto an existing plot
def plot_kde(ax, values, c='black', ls='-', label=None):
    kde = gaussian_kde(values)
    kde.covariance_factor = lambda: 0.15
    kde._compute_covariance()
    xs = np.linspace(np.min(values), np.max(values), 200)
    ys = kde(xs)
    ys /= np.linalg.norm(ys)
    if label == None:
        ax.plot(xs, ys, ls=ls, c=c)
    else:
        ax.plot(xs, ys, ls=ls, c=c, label=label)

# Clean out invalid values from the set
def clean_vals(df):
    df2 = df[df != -np.inf]
    df2 = df2.dropna()
    return df2

# Adds useful reference lines to the plot
def draw_line_references(ax):
    # Significant improvement
    ax.axvline(0.5, ls='-.', c='grey')
    
    # Baselines
    ax.axhline(0, ls=":",  c='lightgrey') 
    ax.axvline(0, ls=":",  c='lightgrey')

# The HRR Equation, for immediate reference within the plot
hirabayashi_equation = r"HRR = $\frac{\mathrm{mJOA (1 Year)} - \mathrm{mJOA (Initial)}}{18 - \mathrm{mJOA (Initial)}}$"

In [ ]:
# Get the HRR for our patients, skipping over initially healthy patients who could not improve whatsoever
hrr_df = mjoa_df.loc[mjoa_df[mjoa_initial_class_key] != "Healthy", hrr_key]

# Generate the initial plot
fig, ax = plt.subplots()

# Plot our reference lines
draw_line_references(ax)

# Plot the distributions by their initial severity class
plot_kde(
    ax, clean_vals(hrr_df[mjoa_df[mjoa_initial_class_key] == 'Severe']), ls='--', c='red', label='Severe'
)
plot_kde(
    ax, clean_vals(hrr_df[mjoa_df[mjoa_initial_class_key] == 'Moderate']), ls='--', c='gold', label='Moderate'
)
plot_kde(
    ax, clean_vals(hrr_df[mjoa_df[mjoa_initial_class_key] == 'Mild']), ls='--', c='green', label='Mild'
)

# Plot the overall distribution
plot_kde(ax, hrr_df, c='blue', label='All')

# Calculate the ratio above and below the HRR significance threshold, and add it
good_ratio = np.sum(hrr_df >= 0.5)/hrr_df.shape[0]
fair_ratio = np.sum(hrr_df < 0.5)/hrr_df.shape[0]

ax.text(0.7, 0.238, f"{good_ratio: .2f}", c='purple')
ax.text(-0.5, 0.238, f"{fair_ratio: .2f}", c='purple')

# Add axis labels
ax.set_xlabel('Hirabayashi Recovery Ratio (HRR)')
ax.set_ylabel('Frequency (Kernel Density Estimate)')

# Add a legend
ax.legend(title='Pre-Surgical DCM Severity')

# Add hirabayashi equation directly to plot
ax.text(-8, 0.15, hirabayashi_equation)

# Add a title
ax.set_title("Distribution of Hirabayashi Recovery Ratio")

plt.tight_layout()

fig.savefig(mjoa_dist_out_path / 'hirabayashi_ratios.svg')

plt.show()

### 